# Neural Network with 1 hidden layer

### Load Modules

In [1]:
# Set the Random Seed
seed_value= 2022

import os
os.environ['PYTHONHASHSEED']=str(seed_value)
import numpy  as np
import pandas  as pd
import datetime
import random
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.formula.api as smf
import tensorflow as tf
from keras.models     import Sequential, load_model
from keras.layers     import Activation, Dense, LeakyReLU
from keras.optimizers import Adam
from keras.callbacks  import EarlyStopping, ModelCheckpoint
from keras.regularizers import l1
from tensorflow.keras.losses import MeanSquaredError

session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

In [3]:
def reset_seeds(seed_value=2022):
    os.environ['PYTHONHASHSEED']=str(seed_value)
    np.random.seed(seed_value)
    random.seed(seed_value)
    tf.random.set_seed(seed_value)
reset_seeds()

### Load Dataset - include RepRisk data

In [4]:
# Load monthly firm characteristics raw data
df = pd.read_parquet('C:/Users/rafae/Documents/HSG/Master Thesis/Data/Final/data07_model_input.parquet')
df = df.sort_values(by=['YM', 'permno'])
df = df.set_index(['year', 'YM', 'permno'])
df

gvkey  reprisk_id         prc         vol         mve_m  \
year YM      permno                                                             
2007 2007-01 10025    11903       37172   45.320000      8086.0  3.700557e+05   
             10026    12825       12684   39.689999      7613.0  7.653725e+05   
             10042    12139        4832    0.720000     26008.0  3.598898e+04   
             10078    12136        1719    6.130000  11333293.0  2.390900e+07   
             10104    12142        4413   16.430000   7234361.0  8.892640e+07   
...                     ...         ...         ...         ...           ...   
2021 2021-12 93304   184167       91339   36.750000    183303.0  1.695898e+06   
             93373   184323       74074    3.020000    528712.0  2.065325e+05   
             93374   184899       64442   74.510002    159495.0  8.587073e+06   
             93423    10567       22547   39.490002    254917.0  3.661156e+06   
             93436   184996       24952  936.719971   6361641.0  1.092218e+09   

                       absacc       acc    aeavol       age       agr  ...  \
year YM      permno                                                    ...   
2007 2007-01 10025   0.698728 -0.745547 -0.646819  0.457506 -0.979644  ...   
             10026   0.577608 -0.635623 -0.393384  0.457506  0.118575  ...   
             10042   0.990840 -0.989822 -0.894148  0.457506 -0.989822  ...   
             10078   0.654962 -0.711959 -0.128753  0.905344 -0.147074  ...   
             10104  -0.014758 -0.107379  0.780153  0.905344  0.770992  ...   
...                       ...       ...       ...       ...       ...  ...   
2021 2021-12 93304  -0.855522  0.706269 -0.871045 -0.500896  0.613134  ...   
             93373  -0.213134  0.143881  0.663284 -0.500896 -0.875821  ...   
             93374  -0.514030  0.423284 -0.223881 -0.500896  0.328955  ...   
             93423   0.242985 -0.303881  0.560597  0.143881 -0.584478  ...   
             93436   0.584478 -0.620299 -0.362388 -0.500896  0.856716  ...   

                     sic2_73  sic2_75  sic2_78  sic2_79  sic2_80  sic2_81  \
year YM      permno                                                         
2007 2007-01 10025         0        0        0        0        0        0   
             10026         0        0        0        0        0        0   
             10042         0        0        0        0        0        0   
             10078         0        0        0        0        0        0   
             10104         1        0        0        0        0        0   
...                      ...      ...      ...      ...      ...      ...   
2021 2021-12 93304         0        0        0        0        0        0   
             93373         0        0        0        0        0        0   
             93374         0        0        0        0        0        0   
             93423         0        0        0        1        0        0   
             93436         0        0        0        0        0        0   

                     sic2_82  sic2_83  sic2_87  sic2_99  
year YM      permno                                      
2007 2007-01 10025         0        0        0        0  
             10026         0        0        0        0  
             10042         0        0        0        0  
             10078         0        0        0        0  
             10104         0        0        0        0  
...                      ...      ...      ...      ...  
2021 2021-12 93304         0        0        0        0  
             93373         0        0        0        0  
             93374         0        0        0        0  
             93423         0        0        0        0  
             93436         0        0        0        0  

[338288 rows x 188 columns]

In [5]:
# Select only relevant columns for X and Y (exclude industry & reprisk rating dummy for now)
info_vars = ['YM', 'year', 'permno', 'gvkey', 'reprisk_id', 'prc', 'vol', 'mve_m']
X_vars = ['absacc', 'acc', 'aeavol', 'age', 'agr', 'baspread', 'beta', 'betasq', 'bm', 'bm_ia', 'cash', 'cashdebt',
          'cashpr', 'cfp', 'cfp_ia', 'chatoia', 'chcsho', 'chempia', 'chinv', 'chmom', 'chpmia', 'chtx', 'cinvest',
          'convind', 'currat', 'depr', 'divi', 'divo', 'dolvol', 'dy', 'ear', 'egr', 'ep', 'gma', 'grcapx', 'grltnoa',
          'herf', 'hire', 'idiovol', 'ill', 'indmom', 'invest', 'lev', 'lgr', 'maxret', 'mom12m', 'mom1m', 'mom36m',
          'mom6m', 'ms', 'mve', 'mve_ia', 'nincr', 'operprof', 'orgcap', 'pchcapx_ia', 'pchcurrat', 'pchdepr',
          'pchgm_pchsale', 'pchquick', 'pchsale_pchinvt', 'pchsale_pchrect', 'pchsale_pchxsga', 'pchsaleinv', 'pctacc',
          'pricedelay', 'ps', 'quick', 'rd', 'rd_mve', 'rd_sale', 'realestate', 'retvol', 'roaq', 'roavol', 'roeq',
          'roic', 'rsup', 'salecash', 'saleinv', 'salerec', 'secured', 'securedind', 'sgr', 'sin', 'sp', 'std_dolvol',
          'std_turn', 'stdacc', 'stdcf', 'tang', 'tb', 'turn', 'zerotrade']
sic2_vars = [col for col in df if col.startswith('sic2')]
reprisk_vars = ['country_sector_average', 'country_sector_average_01', 'current_rri', 'current_rri_01',
                'peak_rri', 'peak_rri_01', 'trend_rri', 'trend_rri_01']
reprisk_rating_vars = [col for col in df if col.startswith('reprisk_rating')]
Y_vars = ['ret', 'ret_wins', 'ret_ex']

# X
X = df[X_vars + sic2_vars + reprisk_vars + reprisk_rating_vars]

# Y
Y = df[['ret_ex']]

X.shape, Y.shape

((338288, 180), (338288, 1))

### Hyperparameter Optimization: 4-Fold CV (12y/4 = 3y) and 3y Test

In [6]:
# Training (12y - 80%) and Test set (3y - 20%)
X_trai = X.loc['2007':'2018']
Y_trai = Y.loc['2007':'2018']

X_test = X.loc['2019':'2021']
Y_test = Y.loc['2019':'2021']

In [7]:
# 4-Fold cross validation (9y training and 3y validation)
K_FOLDs = 4
YEARS = list(X_trai.index.unique(level='year').astype('str')) 
TOT = len(YEARS)
TRA = int(TOT* (K_FOLDs-1) / K_FOLDs)
OFF = TOT - TRA

for FOLD in range(K_FOLDs):
    VALI = YEARS[(FOLD*OFF):((FOLD+1)*OFF)]
    TRAI = [x for x in YEARS if x not in VALI]
    print(VALI, TRAI)    

['2007', '2008', '2009'] ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']
['2010', '2011', '2012'] ['2007', '2008', '2009', '2013', '2014', '2015', '2016', '2017', '2018']
['2013', '2014', '2015'] ['2007', '2008', '2009', '2010', '2011', '2012', '2016', '2017', '2018']
['2016', '2017', '2018'] ['2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']


In [8]:
# Model
model_name = 'NN1_esg'

def create_NN1(l_rate=0.01, l1_pen=0.0001):
    model = Sequential()
    model.add(Dense(32,
                    activation='relu',
                    input_dim=X_trai.shape[1],
                    kernel_regularizer=l1(l1_pen)))
    model.add(Dense(1))
    model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=l_rate))
    return model

In [9]:
# Hyperparameter optimization
Y_val_preds = Y_trai.copy()
Y_test_preds = Y_test.copy()
results = []

earlyStopping  = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')

model_ix = 0
for l1_pen in [0.001, 0.0005, 0.0001, 0.00005, 0.00001]:
        for l_rate in [0.01, 0.005, 0.001]:
            for ensemble in list(range(0,10)):
                print("•", model_ix, ':',  end=' ')
                    
                for FOLD in range(K_FOLDs):
                    VALI = YEARS[(FOLD*OFF):((FOLD+1)*OFF)]
                    TRAI = [x for x in YEARS if x not in VALI]
                        
                    mcp_save = ModelCheckpoint(
                        '%s/%s_%d_%d.h5'%(model_name, model_name, model_ix, FOLD),
                        save_best_only=True, monitor='val_loss', mode='min', save_weights_only=True
                    )

                    reset_seeds(ensemble*10000)
                        
                    NN_model = create_NN1(l_rate, l1_pen)
                        
                    history = NN_model.fit(
                        X_trai.loc[TRAI], Y_trai.loc[TRAI],
                        validation_data=(X_trai.loc[VALI], Y_trai.loc[VALI]),
                        epochs=200, verbose=0, shuffle=True, batch_size=2000,
                        callbacks=[earlyStopping, mcp_save]
                    )
                    tra_loss = np.min(history.history['loss'])
                    val_loss = np.min(history.history['val_loss'])

                    results.append({
                        'model_ix' :model_ix,
                        'l_rate'   :l_rate,
                        'l1_pen'   :l1_pen,
                        'fold'     :FOLD,
                        'tra_loss' :tra_loss,
                        'val_loss' :val_loss
                    })
                    
                    # Calculate validation predictions
                    best_model = create_NN1(l_rate, l1_pen)
                    best_model.load_weights(r'%s/%s_%d_%d.h5'%(model_name, model_name, model_ix, FOLD));
                    Y_val_preds.loc[VALI, '%s_%d'%(model_name, model_ix)] = best_model.predict(X_trai.loc[VALI])
                    
                    # Calculate predictions for test data, if FOLD = 0
                    if FOLD==0:
                        Y_test_preds.loc[['2019','2020', '2021'], '%s_%d'%(model_name, model_ix)] = best_model.predict(X_test)

                model_ix += 1

• 0 : Epoch 7: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 7: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 7: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 7: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 1 : Epoch 8: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 8: early stopping
2193/2193 [==============================] - 5s 2ms/step
Epoch 8: early stopping
2156/2156 [==============================] - 6s 3ms/step
Epoch 8: early stopping
2066/2066 [==============================] - 5s 2ms/step
• 2 : Epoch 7: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 8: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 8: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 7: early stopping
2066/2066 [==============================] - 5s 2ms/step
• 3 : Epoc

1929/1929 [==============================] - 4s 2ms/step
Epoch 12: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 12: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 13: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 43 : Epoch 9: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 9: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 9: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 11: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 44 : Epoch 9: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 8: early stopping
2193/2193 [==============================] - 5s 2ms/step
Epoch 8: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 8: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 45 : Epoch 12: early stopping
19

1929/1929 [==============================] - 3s 2ms/step
Epoch 15: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 14: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 11: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 64 : Epoch 15: early stopping
1929/1929 [==============================] - 3s 2ms/step
Epoch 15: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 12: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 12: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 65 : Epoch 12: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 18: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 15: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 11: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 66 : Epoch 11: early stop

C:\Users\rafae\AppData\Local\Temp\ipykernel_13880\2749122029.py:48: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Y_val_preds.loc[VALI, '%s_%d'%(model_name, model_ix)] = best_model.predict(X_trai.loc[VALI])


1929/1929 [==============================] - 4s 2ms/step


...

Epoch 60: early stopping
2193/2193 [==============================] - 5s 2ms/step
Epoch 56: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 61: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 149 : Epoch 44: early stopping
   1/1929 [..............................] - ETA: 30s

C:\Users\rafae\AppData\Local\Temp\ipykernel_13880\2749122029.py:48: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Y_val_preds.loc[VALI, '%s_%d'%(model_name, model_ix)] = best_model.predict(X_trai.loc[VALI])


1929/1929 [==============================] - 4s 2ms/step


C:\Users\rafae\AppData\Local\Temp\ipykernel_13880\2749122029.py:52: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Y_test_preds.loc[['2019','2020', '2021'], '%s_%d'%(model_name, model_ix)] = best_model.predict(X_test)


Epoch 52: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 47: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 47: early stopping
2066/2066 [==============================] - 4s 2ms/step


In [10]:
# Save Y_val_preds
Y_val_preds.to_csv(r'%s/%s_val_preds.csv'%(model_name, model_name))
Y_val_preds

ret_ex  NN1_esg_0  NN1_esg_1  NN1_esg_2  NN1_esg_3  \
year YM      permno                                                         
2007 2007-01 10025  -0.031894   0.008402   0.009199   0.010144   0.009644   
             10026  -0.042317   0.008402   0.009199   0.009170   0.009644   
             10042  -0.125751   0.008402   0.009199   0.014780   0.009644   
             10078  -0.080607   0.008402   0.009199   0.009170   0.009644   
             10104  -0.046341   0.008402   0.009199   0.008994   0.009644   
...                       ...        ...        ...        ...        ...   
2018 2018-12 93420   0.086508   0.006446   0.005313   0.006006   0.003000   
             93422   0.466817   0.006446   0.005313   0.006006   0.003000   
             93423   0.105036   0.006446   0.005313   0.006006   0.003000   
             93429  -0.048712   0.006446   0.005313   0.008891   0.003000   
             93436  -0.079564   0.006446   0.005313   0.006006   0.003000   

                     NN1_esg_4  NN1_esg_5  NN1_esg_6  NN1_esg_7  NN1_esg_8  \
year YM      permno                                                          
2007 2007-01 10025    0.010523   0.009657   0.010455   0.009555   0.008813   
             10026    0.010523   0.009657   0.010455   0.009555   0.008813   
             10042    0.010523   0.009657   0.010455   0.009555   0.008813   
             10078    0.010523   0.009657   0.010455   0.009555   0.008813   
             10104    0.010523   0.009657   0.010455   0.009555   0.008813   
...                        ...        ...        ...        ...        ...   
2018 2018-12 93420    0.003772   0.006600   0.006356   0.006461   0.005495   
             93422    0.003772   0.006600   0.006356   0.006461   0.005495   
             93423    0.003772   0.006600   0.006356   0.006461   0.005495   
             93429    0.003772   0.006600   0.006356   0.006461   0.005495   
             93436    0.003772   0.006600   0.006356   0.006461   0.005495   

                     ...  NN1_esg_140  NN1_esg_141  NN1_esg_142  NN1_esg_143  \
year YM      permno  ...                                                       
2007 2007-01 10025   ...     0.022350     0.019942     0.040869     0.019536   
             10026   ...     0.013079    -0.002314    -0.002577     0.010872   
             10042   ...    -0.018772    -0.006784    -0.042111    -0.012971   
             10078   ...     0.004890     0.002270     0.008557     0.011027   
             10104   ...     0.028376     0.018033     0.019897     0.012753   
...                  ...          ...          ...          ...          ...   
2018 2018-12 93420   ...    -0.016003    -0.005728    -0.016413     0.020196   
             93422   ...    -0.040303     0.012946    -0.022306    -0.033589   
             93423   ...    -0.009459     0.007143     0.009294    -0.007636   
             93429   ...     0.012960     0.024905     0.019805     0.022287   
             93436   ...     0.012097     0.010542     0.025530    -0.001984   

                     NN1_esg_144  NN1_esg_145  NN1_esg_146  NN1_esg_147  \
year YM      permno                                                       
2007 2007-01 10025      0.015804     0.033893     0.021692     0.027074   
             10026      0.012494     0.005684     0.004576     0.014654   
             10042     -0.023156    -0.025220    -0.015538    -0.020080   
             10078      0.008617     0.014806    -0.001504     0.009864   
             10104      0.018037     0.005040     0.011546     0.012472   
...                          ...          ...          ...          ...   
2018 2018-12 93420     -0.029813    -0.023298     0.003063    -0.024242   
             93422     -0.038602    -0.003587    -0.014733     0.002304   
             93423      0.008662     0.004341     0.000903    -0.005420   
             93429      0.007167     0.009104     0.005399     0.029118   
             93436      0.003258    -0.007285     0.011493    -0.

In [11]:
# Save Y_test_preds
Y_test_preds.to_csv(r'%s/%s_test_preds.csv'%(model_name, model_name))
Y_test_preds

ret_ex  NN1_esg_0  NN1_esg_1  NN1_esg_2  NN1_esg_3  \
year YM      permno                                                         
2019 2019-01 10026   0.004225   0.008402   0.009438    0.00917   0.009644   
             10104   0.036026   0.008402   0.009199    0.00917   0.009644   
             10107   0.075381   0.008402   0.009199    0.00917   0.009644   
             10138   0.072777   0.008402   0.009199    0.00917   0.009644   
             10145   0.076596   0.008402   0.009199    0.00917   0.009644   
...                       ...        ...        ...        ...        ...   
2021 2021-12 93304  -0.096386   0.008402   0.009199    0.00917   0.009644   
             93373  -0.019481   0.008402   0.009199    0.00917   0.009644   
             93374  -0.047552   0.008402   0.009199    0.00917   0.009644   
             93423  -0.072569   0.008402   0.009199    0.00917   0.009644   
             93436  -0.113609   0.008402   0.009199    0.00917   0.009644   

                     NN1_esg_4  NN1_esg_5  NN1_esg_6  NN1_esg_7  NN1_esg_8  \
year YM      permno                                                          
2019 2019-01 10026    0.010523   0.009657   0.010455   0.009555   0.008813   
             10104    0.010523   0.009657   0.010455   0.009555   0.008813   
             10107    0.010523   0.009657   0.010455   0.009555   0.008813   
             10138    0.010523   0.009657   0.010455   0.009555   0.008813   
             10145    0.010523   0.009657   0.010455   0.009555   0.008813   
...                        ...        ...        ...        ...        ...   
2021 2021-12 93304    0.010523   0.009657   0.010455   0.009555   0.008813   
             93373    0.010523   0.009657   0.010455   0.009555   0.008813   
             93374    0.010523   0.009657   0.010455   0.009555   0.008813   
             93423    0.010523   0.009657   0.010455   0.009555   0.008813   
             93436    0.010523   0.009657   0.010455   0.009555   0.008813   

                     ...  NN1_esg_140  NN1_esg_141  NN1_esg_142  NN1_esg_143  \
year YM      permno  ...                                                       
2019 2019-01 10026   ...     0.007003     0.008438     0.005607     0.012038   
             10104   ...     0.019544     0.011082     0.009421     0.008697   
             10107   ...     0.023354     0.016375     0.015752     0.015466   
             10138   ...     0.007643     0.013443     0.005030     0.014818   
             10145   ...     0.021212     0.004975     0.018237     0.006123   
...                  ...          ...          ...          ...          ...   
2021 2021-12 93304   ...     0.018660     0.006989     0.009709     0.017751   
             93373   ...    -0.039067    -0.009118     0.001682    -0.017144   
             93374   ...     0.008227     0.008774     0.009163     0.012038   
             93423   ...     0.005553     0.005734     0.003426     0.004395   
             93436   ...     0.014948     0.017483    -0.003957     0.003816   

                     NN1_esg_144  NN1_esg_145  NN1_esg_146  NN1_esg_147  \
year YM      permno                                                       
2019 2019-01 10026      0.008418     0.008864     0.007735     0.003667   
             10104      0.005148     0.004837     0.005946     0.009027   
             10107      0.016334     0.006908     0.005813     0.011745   
             10138      0.013806     0.007514     0.008837     0.010366   
             10145      0.009519     0.008758     0.005343     0.010487   
...                          ...          ...          ...          ...   
2021 2021-12 93304      0.023575     0.004844     0.023308     0.019024   
             93373     -0.012193    -0.046751    -0.007877    -0.001576   
             93374      0.014154    -0.002042     0.009898     0.003241   
             93423      0.031617     0.017748     0.012101    -0.004999   
             93436      0.012482    -0.000534     0.000843    -0.

In [12]:
# Result overview
table = pd.DataFrame(results)
table = table.groupby(['model_ix', 'l_rate', 'l1_pen']).mean().sort_values('val_loss')
table.to_csv(r'%s/%s_results.csv'%(model_name, model_name))

np.sqrt(table[['tra_loss', 'val_loss']].head(20)) * 100

,,,tra_loss,val_loss
model_ix,l_rate,l1_pen,,
86,0.001,0.00010,12.987796,12.977936
84,0.001,0.00010,12.984003,12.978046
89,0.001,0.00010,12.984608,12.981751
85,0.001,0.00010,12.985440,12.982331
83,0.001,0.00010,12.986126,12.982507
109,0.005,0.00005,13.006018,12.985662
80,0.001,0.00010,12.988146,12.985959
87,0.001,0.00010,12.991783,12.990951
88,0.001,0.00010,12.990874,12.993499


## Model Selection: Maximize SR on LS-Portfolio on validation data

In [13]:
# Model
model_name = 'NN1_esg'

# Get predictions
val = pd.read_csv(r'%s/%s_val_preds.csv'%(model_name, model_name), index_col=['year', 'YM', 'permno'])
test = pd.read_csv(r'%s/%s_test_preds.csv'%(model_name, model_name), index_col=['year', 'YM', 'permno'])

# Get overview table
table = pd.read_csv(r'%s/%s_results.csv'%(model_name, model_name)).sort_values(['model_ix'])

# Extract hyperparameters
list_l_rate = list(pd.unique(table['l_rate']))
list_l1_pen = list(pd.unique(table['l1_pen']))

In [14]:
# Loop over hyperparameter combinations to get ensemble predictions for each hyperparameter combination
val_results = []
hyper_comb = 0
for l_rate in list_l_rate:
    for l1_pen in list_l1_pen:
        # Get list of relevant model_ix
        list_model_ix = table[(table['l_rate'] == l_rate) & (table['l1_pen'] == l1_pen)]
        list_model_ix = list(list_model_ix['model_ix'])
        # Get list of column names
        list_col_names = []
        for ele in list_model_ix:
            list_col_names.append('%s_%d'%(model_name, ele))
        # Select relevant returns and average predictions
        val_ret = val[list_col_names]
        val_ret = val_ret.mean(axis=1)
        # Append ensemble return prediction to true returns
        comb = val[['ret_ex']].copy()
        comb['ret_pred'] = val_ret
        comb = comb.reset_index()
        # Sort the data by predicted returns and divide the data into quintiles
        comb['quintile'] = comb.groupby(['YM'])['ret_pred'].transform(lambda x: pd.qcut(x.rank(method='first'), 5, labels=np.arange(1,6)))
        # Calculate the mean return for each YM
        comb_mean = comb.groupby(['YM', 'quintile']).agg(ret_ex = ('ret_ex', 'mean'))
        # Add LS-Strategy
        comb_mean = comb_mean[['ret_ex']].unstack().add_prefix('Q')
        comb_mean.columns = comb_mean.columns.droplevel(0)
        comb_mean.columns.name = None
        comb_mean['LS'] = comb_mean['Q5'] - comb_mean['Q1']
        # Calculate the average return, standard deviation and Sharpe Ratio (annualized) per Quintile
        summary = pd.DataFrame()
        summary['mean'] = comb_mean.mean()
        summary['std'] = comb_mean.std()
        summary['SR'] = summary['mean'] / summary['std'] * np.sqrt(12)
        # Append results
        val_results.append({
            'hyper_comb':hyper_comb,
            'l_rate'    :l_rate,
            'l1_pen'    :l1_pen,
            'SR_Q1'     :summary.loc['Q1','SR'],
            'SR_Q5'     :summary.loc['Q5','SR'],
            'SR_LS'     :summary.loc['LS','SR'],
            'Mean_Q1'   :summary.loc['Q1','mean'],
            'Mean_Q5'   :summary.loc['Q5','mean'],
            'Mean_LS'   :summary.loc['LS','mean'],
        })
        hyper_comb += 1
        
# Save results
val_results = pd.DataFrame(val_results)
val_results.to_csv(r'results/{}_val.csv'.format(model_name))

In [15]:
# Select best hyperparemeters (max. SR_LS)
val_results = val = pd.read_csv(r'results/{}_val.csv'.format(model_name))
val_results = val_results.set_index(['l_rate', 'l1_pen']).sort_values(['SR_LS'], ascending=False)
l_rate_opt, l1_pen_opt = tuple([x for x in val_results.iloc[0].name[0:]])
print('Optimal l_rate:', l_rate_opt)
print('Optimal l1_pen:', l1_pen_opt)
val_results

Optimal l_rate: 0.001
Optimal l1_pen: 1e-05


Unnamed: 0  hyper_comb     SR_Q1     SR_Q5     SR_LS  \
l_rate l1_pen                                                          
0.001  0.00001          14          14  0.063368  0.498897  1.188718   
0.005  0.00001           9           9  0.071078  0.484416  0.908946   
0.001  0.00005          13          13  0.026355  0.525324  0.742640   
0.005  0.00005           8           8  0.011884  0.483424  0.643707   
0.010  0.00001           4           4  0.068596  0.432467  0.632907   
       0.00005           3           3  0.013391  0.573711  0.628919   
0.001  0.00010          12          12  0.043477  0.552516  0.607630   
       0.00050          11          11 -0.015002  0.657225  0.598854   
       0.00100          10          10 -0.004455  0.647257  0.567694   
0.010  0.00010           2           2  0.038252  0.559870  0.552725   
0.005  0.00100           5           5  0.140703  0.666838  0.551687   
       0.00010           7           7  0.018122  0.606400  0.527393   
       0.00050           6           6  0.038109  0.628818  0.478412   
0.010  0.00100           0           0  0.372781  0.376058  0.210034   
       0.00050           1           1  0.300150  0.321782 -0.071134   

                 Mean_Q1   Mean_Q5   Mean_LS  
l_rate l1_pen                                 
0.001  0.00001  0.001288  0.009306  0.008018  
0.005  0.00001  0.001534  0.008507  0.006973  
0.001  0.00005  0.000610  0.008722  0.008112  
0.005  0.00005  0.000288  0.007822  0.007534  
0.010  0.00001  0.001551  0.007383  0.005832  
       0.00005  0.000327  0.008479  0.008153  
0.001  0.00010  0.001061  0.008646  0.007585  
       0.00050 -0.000360  0.008506  0.008867  
       0.00100 -0.000107  0.008291  0.008399  
0.010  0.00010  0.000942  0.008386  0.007444  
0.005  0.00100  0.002999  0.008956  0.005957  
       0.00010  0.000446  0.008339  0.007893  
       0.00050  0.000956  0.008197  0.007241  
0.010  0.00100  0.005912  0.007002  0.001090  
       0.00050  0.006053  0.005468 -0.000585

## Out-of-Sample: Get Predictions of model with best hyperparameter combination

In [16]:
# Get list of model_ix's with best hyperparameter combinations and make ensemble predictions
list_model_ix = table[(table['l_rate'] == l_rate_opt) & (table['l1_pen'] == l1_pen_opt)]
list_model_ix = list(list_model_ix['model_ix'])

# Get list of column names
list_col_names = []
for ele in list_model_ix:
    list_col_names.append('%s_%d'%(model_name, ele))
    
# Select relevant returns and average predictions
test_ret = test[list_col_names].copy()
test_ret['y_pred'] = test_ret.mean(axis=1)

# Save predictions
test_ret = test_ret.reset_index()
test_ret = test_ret[['YM', 'permno', 'y_pred']]
test_ret.to_csv(r'results/{}_predictions.csv'.format(model_name), index=False)